In [1]:
import polars as pl
import pandas as pd
import plotly.express as px 
import pyarrow


In [2]:
lf = pl.scan_csv('D:\Base_de_dados_anvisa\years_active principle.csv')

lf.head().collect()

PRINCIPIO_ATIVO,UF_VENDA,DATA,count
str,str,i64,i64
"""ESTOLATO DE ER...","""SP""",102014,5948
"""CLORIDRATO DE ...","""SP""",102014,5885
"""ESTOLATO DE ER...","""SP""",72014,5809
"""CLORIDRATO DE ...","""SP""",92014,5696
"""CLORIDRATO DE ...","""SP""",72014,5618


In [9]:
"""     data_col = data_col.with_column()
    data_col = data_col.apply(lambda x: x.zfill(8))
    data_col = data_col.str.replace('00','01')
    data_col = pd.to_datetime(data_col, format='%d%m%Y') """

"     data_col = data_col.with_column()\n    data_col = data_col.apply(lambda x: x.zfill(8))\n    data_col = data_col.str.replace('00','01')\n    data_col = pd.to_datetime(data_col, format='%d%m%Y') "

In [10]:
def correct_data(LazyFrame, col_name):
    z=8
    q = (
        lf
        .with_columns(
            pl.col(col_name)
            .cast(pl.Utf8)
        )
        .with_columns(
            pl.when(
                pl.col(col_name).cast(pl.Utf8).str.lengths() > z
            )
            .then(
                pl.col(col_name).cast(pl.Utf8)
            )
            .otherwise(
                pl.concat_str([pl.lit("0" * z), pl.col(col_name).cast(pl.Utf8)]).str.slice(-z)
            )
            .alias("DATETIME")
        )
        .with_columns(
            pl.col('DATETIME').str.replace('00','01').alias('DATETIME')
        )
        .with_columns(
            pl.col('DATETIME').str.strptime(pl.Date, fmt='%d%m%Y').cast(pl.Datetime)
        )
        .sort(by=pl.col('DATETIME'))

    )
    df = q.collect()
    
    return df


In [14]:
df = correct_data(lf, 'DATA')
df

PRINCIPIO_ATIVO,UF_VENDA,DATA,count,DATETIME
str,str,str,i64,datetime[μs]
"""CLORIDRATO DE ...","""SP""","""12014""",5055,2014-01-01 00:00:00
"""CLORIDRATO DE ...","""SP""","""22014""",5073,2014-02-01 00:00:00
"""CLORIDRATO DE ...","""SP""","""32014""",5174,2014-03-01 00:00:00
"""CLORIDRATO DE ...","""SP""","""42014""",5241,2014-04-01 00:00:00
"""ESTOLATO DE ER...","""SP""","""52014""",5494,2014-05-01 00:00:00
"""CLORIDRATO DE ...","""SP""","""52014""",5340,2014-05-01 00:00:00
"""CLORIDRATO DE ...","""SP""","""62014""",5175,2014-06-01 00:00:00
"""ESTOLATO DE ER...","""SP""","""72014""",5809,2014-07-01 00:00:00
"""CLORIDRATO DE ...","""SP""","""72014""",5618,2014-07-01 00:00:00


In [15]:
#px.line(df, x=df.select('DATETIME').to_series(), y=df.select('count').to_series(), color=df.select('PRINCIPIO_ATIVO').to_series())
teste = df.to_pandas()
px.line(teste, x='DATETIME', y='count', color='PRINCIPIO_ATIVO')

In [ ]:
df.select('PRINCIPIO_ATIVO').to_series()

PRINCIPIO_ATIVO
str
"""ESTOLATO DE ER..."
"""CLORIDRATO DE ..."
"""ESTOLATO DE ER..."
"""CLORIDRATO DE ..."
"""CLORIDRATO DE ..."
"""ESTOLATO DE ER..."
"""CLORIDRATO DE ..."
"""CLORIDRATO DE ..."
"""CLORIDRATO DE ..."
